# Architectural Style Generation

In [ ]:
__author__ = "Caroline Ho and Cole Thomson"
__version__ = "CS230, Stanford, Autumn 2018 term"

## Contents
1. [Overview](#Overview)
2. [Set-Up](#Set-Up)
3. [Data](#Data)
4. [Model](#Model)
5. [Run Model](#Run-Model)
6. [Resources](#Resources)

## Overview

In this notebook, we use conditional GANs to generate images of buildings with given architectural styles.

## Set-Up

Run the following cells to import necessary libraries/functions and set global variables.

In [1]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio

In [2]:
# Parameters
image_size = 64
label_dim = 25
G_input_dim = 625
G_output_dim = 3
D_input_dim = 3
D_output_dim = 1
num_filters = [1024, 512, 256, 128]

learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 4
num_epochs = 20
save_dir = 'results/'

## Data

Download [Xu et al.'s architecture dataset](https://drive.google.com/file/d/0Bwo0SFiZwl3JVGRlWGZUaW5va00/edit) and place it in the current directory.

This dataset contains 4,843 images of buildings from 25 architecture style classes ranging from Achaemenid to Tudor Revival. Image dimensions/aspect ratios are not consistent.

In [3]:
transform = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),std=(0.5, 0.5, 0.5))
])

data = dsets.ImageFolder('arcDataset', transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=data,
                                          batch_size=batch_size,
                                          shuffle=True)

## Helper Functions

In [4]:
def to_var(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

In [5]:
# De-normalization
def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

## Model

In [6]:
# Generator model
class Generator(torch.nn.Module):
    def __init__(self, input_dim, label_dim, num_filters, output_dim):
        super(Generator, self).__init__()

        # Hidden layers
        self.hidden_layer1 = torch.nn.Sequential()
        self.hidden_layer2 = torch.nn.Sequential()
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Deconvolutional layer
            if i == 0:
                # For input
                input_deconv = torch.nn.ConvTranspose2d(input_dim, int(num_filters[i]/2), kernel_size=4, stride=1, padding=0)
                self.hidden_layer1.add_module('input_deconv', input_deconv)

                # Initializer
                torch.nn.init.normal_(input_deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(input_deconv.bias, 0.0)

                # Batch normalization
                self.hidden_layer1.add_module('input_bn', torch.nn.BatchNorm2d(int(num_filters[i]/2)))

                # Activation
                self.hidden_layer1.add_module('input_act', torch.nn.ReLU())

                # For label
                label_deconv = torch.nn.ConvTranspose2d(label_dim, int(num_filters[i]/2), kernel_size=4, stride=1, padding=0)
                self.hidden_layer2.add_module('label_deconv', label_deconv)

                # Initializer
                torch.nn.init.normal_(label_deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(label_deconv.bias, 0.0)

                # Batch normalization
                self.hidden_layer2.add_module('label_bn', torch.nn.BatchNorm2d(int(num_filters[i]/2)))

                # Activation
                self.hidden_layer2.add_module('label_act', torch.nn.ReLU())
            else:
                deconv = torch.nn.ConvTranspose2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

                deconv_name = 'deconv' + str(i + 1)
                self.hidden_layer.add_module(deconv_name, deconv)

                # Initializer
                torch.nn.init.normal_(deconv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(deconv.bias, 0.0)

                # Batch normalization
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

                # Activation
                act_name = 'act' + str(i + 1)
                self.hidden_layer.add_module(act_name, torch.nn.ReLU())

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Deconvolutional layer
        out = torch.nn.ConvTranspose2d(num_filters[i], output_dim, kernel_size=4, stride=2, padding=1)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal_(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant_(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Tanh())

    def forward(self, z, c):
        h1 = self.hidden_layer1(z)
        h2 = self.hidden_layer2(c)
        x = torch.cat([h1, h2], 1)
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

In [7]:
# Discriminator model
class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, label_dim, num_filters, output_dim):
        super(Discriminator, self).__init__()

        self.hidden_layer1 = torch.nn.Sequential()
        self.hidden_layer2 = torch.nn.Sequential()
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Convolutional layer
            if i == 0:
                # For input
                input_conv = torch.nn.Conv2d(input_dim, int(num_filters[i]/2), kernel_size=4, stride=2, padding=1)
                self.hidden_layer1.add_module('input_conv', input_conv)

                # Initializer
                torch.nn.init.normal_(input_conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(input_conv.bias, 0.0)

                # Activation
                self.hidden_layer1.add_module('input_act', torch.nn.LeakyReLU(0.2))

                # For label
                label_conv = torch.nn.Conv2d(label_dim, int(num_filters[i]/2), kernel_size=4, stride=2, padding=1)
                self.hidden_layer2.add_module('label_conv', label_conv)

                # Initializer
                torch.nn.init.normal_(label_conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(label_conv.bias, 0.0)

                # Activation
                self.hidden_layer2.add_module('label_act', torch.nn.LeakyReLU(0.2))
            else:
                conv = torch.nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

                conv_name = 'conv' + str(i + 1)
                self.hidden_layer.add_module(conv_name, conv)

                # Initializer
                torch.nn.init.normal_(conv.weight, mean=0.0, std=0.02)
                torch.nn.init.constant_(conv.bias, 0.0)

                # Batch normalization
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

                # Activation
                act_name = 'act' + str(i + 1)
                self.hidden_layer.add_module(act_name, torch.nn.LeakyReLU(0.2))

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Convolutional layer
        out = torch.nn.Conv2d(num_filters[i], output_dim, kernel_size=4, stride=1, padding=0)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal_(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant_(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Sigmoid())

    def forward(self, z, c):
        h1 = self.hidden_layer1(z)
        h2 = self.hidden_layer2(c)
        x = torch.cat([h1, h2], 1)
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out

## Plotting Functions

In [8]:
# Plot losses
def plot_loss(d_losses, g_losses, num_epoch, save=False, save_dir='results/', show=False):
    fig, ax = plt.subplots()
    ax.set_xlim(0, num_epochs)
    ax.set_ylim(0, max(np.max(g_losses), np.max(d_losses))*1.1)
    plt.xlabel('Epoch {0}'.format(num_epoch + 1))
    plt.ylabel('Loss values')
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.legend()

    # save figure
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'cDCGAN_losses_epoch_{:d}'.format(num_epoch + 1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

In [9]:
def plot_result(generator, noise, label, num_epoch, save=False, save_dir='results/', show=False, fig_size=(100, 100)):
    generator.eval()

    noise = Variable(noise.cuda())
    label = Variable(label.cuda())
    gen_image = generator(noise, label)
    gen_image = denorm(gen_image)

    generator.train()

    n_rows = np.sqrt(noise.size()[0]).astype(np.int32)
    n_cols = np.sqrt(noise.size()[0]).astype(np.int32)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    for ax, img in zip(axes.flatten(), gen_image):
        ax.axis('off')
        ax.set_adjustable('box-forced')
        # Scale to 0-255
        img = (((img - img.min()) * 255) / (img.max() - img.min())).cpu().data.numpy().transpose(1, 2, 0).astype(
            np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch + 1)
    fig.text(0.5, 0.04, title, ha='center')

    # save figure
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'cDCGAN_epoch_{:d}'.format(num_epoch+1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()

## Run Model

In [10]:
G = Generator(G_input_dim, label_dim, num_filters, G_output_dim)
D = Discriminator(D_input_dim, label_dim, num_filters[::-1], D_output_dim)
G.cuda()
D.cuda()

Discriminator(
  (hidden_layer1): Sequential(
    (input_conv): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (input_act): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer2): Sequential(
    (label_conv): Conv2d(25, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (label_act): LeakyReLU(negative_slope=0.2)
  )
  (hidden_layer): Sequential(
    (conv2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): LeakyReLU(negative_slope=0.2)
    (conv3): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn3): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act3): LeakyReLU(negative_slope=0.2)
    (conv4): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (bn4): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act4): LeakyRe

In [11]:
# Loss function
criterion = torch.nn.BCELoss()

# Optimizers
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=betas)

In [12]:
# label preprocess
onehot = torch.zeros(label_dim, label_dim)
onehot = onehot.scatter_(1, torch.LongTensor(list(range(label_dim))).view(label_dim, 1), 1).view(label_dim, label_dim, 1, 1)
fill = torch.zeros([label_dim, label_dim, image_size, image_size])
for i in range(label_dim):
    fill[i, i, :, :] = 1

In [13]:
# Fixed noise & label for test
temp_noise = torch.randn(label_dim, G_input_dim)
fixed_noise = temp_noise
fixed_c = torch.zeros(label_dim, 1)
for i in range(label_dim - 1):
    fixed_noise = torch.cat([fixed_noise, temp_noise], 0)
    temp = torch.ones(label_dim, 1) + i
    fixed_c = torch.cat([fixed_c, temp], 0)

fixed_noise = fixed_noise.view(-1, G_input_dim, 1, 1)
fixed_label = torch.zeros(G_input_dim, label_dim)
fixed_label.scatter_(1, fixed_c.type(torch.LongTensor), 1)
fixed_label = fixed_label.view(-1, label_dim, 1, 1)

In [14]:
# Training GAN
D_avg_losses = []
G_avg_losses = []

step = 0
for epoch in range(num_epochs):
    D_losses = []
    G_losses = []

    if epoch == 5 or epoch == 10:
        G_optimizer.param_groups[0]['lr'] /= label_dim
        D_optimizer.param_groups[0]['lr'] /= label_dim

    # minibatch training
    for i, (images, labels) in enumerate(data_loader):

        # image data
        mini_batch = images.size()[0]
        x_ = Variable(images.cuda())

        # labels
        y_real_ = Variable(torch.ones(mini_batch).cuda())
        y_fake_ = Variable(torch.zeros(mini_batch).cuda())
        c_fill_ = Variable(fill[labels].cuda())

        # Train discriminator with real data
        D_real_decision = D(x_, c_fill_).squeeze()
        D_real_loss = criterion(D_real_decision, y_real_)

        # Train discriminator with fake data
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())

        c_ = (torch.rand(mini_batch, 1) * label_dim).type(torch.LongTensor).squeeze()
        c_onehot_ = Variable(onehot[c_].cuda())
        gen_image = G(z_, c_onehot_)

        c_fill_ = Variable(fill[c_].cuda())
        D_fake_decision = D(gen_image, c_fill_).squeeze()
        D_fake_loss = criterion(D_fake_decision, y_fake_)

        # Back propagation
        D_loss = D_real_loss + D_fake_loss
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())

        c_ = (torch.rand(mini_batch, 1) * label_dim).type(torch.LongTensor).squeeze()
        c_onehot_ = Variable(onehot[c_].cuda())
        gen_image = G(z_, c_onehot_)

        c_fill_ = Variable(fill[c_].cuda())
        D_fake_decision = D(gen_image, c_fill_).squeeze()
        G_loss = criterion(D_fake_decision, y_real_)

        # Back propagation
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(torch.Tensor.item(D_loss.data))
        G_losses.append(torch.Tensor.item(G_loss.data))

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(data_loader), torch.Tensor.item(D_loss.data), torch.Tensor.item(G_loss.data)))
        step += 1

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    plot_loss(D_avg_losses, G_avg_losses, epoch, save=True, save_dir=save_dir)

    # Show result for fixed noise
    plot_result(G, fixed_noise, fixed_label, epoch, save=True, save_dir=save_dir)

# Make gif
loss_plots = []
gen_image_plots = []
for epoch in range(num_epochs):
    # plot for generating gif
    save_fn1 = save_dir + 'cDCGAN_losses_epoch_{:d}'.format(epoch + 1) + '.png'
    loss_plots.append(imageio.imread(save_fn1))

    save_fn2 = save_dir + 'cDCGAN_epoch_{:d}'.format(epoch + 1) + '.png'
    gen_image_plots.append(imageio.imread(save_fn2))

imageio.mimsave(save_dir + 'cDCGAN_losses_epochs_{:d}'.format(num_epochs) + '.gif', loss_plots, fps=5)
imageio.mimsave(save_dir + 'cDCGAN_epochs_{:d}'.format(num_epochs) + '.gif', gen_image_plots, fps=5)

Epoch [1/20], Step [1/1199], D_loss: 1.3777, G_loss: 2.1853
Epoch [1/20], Step [2/1199], D_loss: 1.0871, G_loss: 1.5253
Epoch [1/20], Step [3/1199], D_loss: 1.1494, G_loss: 2.5125
Epoch [1/20], Step [4/1199], D_loss: 1.0148, G_loss: 2.8835
Epoch [1/20], Step [5/1199], D_loss: 0.3445, G_loss: 2.8756
Epoch [1/20], Step [6/1199], D_loss: 1.0807, G_loss: 3.2679
Epoch [1/20], Step [7/1199], D_loss: 0.1711, G_loss: 2.0963
Epoch [1/20], Step [8/1199], D_loss: 1.2725, G_loss: 3.0511
Epoch [1/20], Step [9/1199], D_loss: 2.5382, G_loss: 2.0810
Epoch [1/20], Step [10/1199], D_loss: 2.8085, G_loss: 2.7656
Epoch [1/20], Step [11/1199], D_loss: 2.1746, G_loss: 2.6669
Epoch [1/20], Step [12/1199], D_loss: 0.6270, G_loss: 3.7012
Epoch [1/20], Step [13/1199], D_loss: 0.6180, G_loss: 2.4309
Epoch [1/20], Step [14/1199], D_loss: 0.6776, G_loss: 2.0067
Epoch [1/20], Step [15/1199], D_loss: 1.0401, G_loss: 2.5940
Epoch [1/20], Step [16/1199], D_loss: 1.3821, G_loss: 5.3984
Epoch [1/20], Step [17/1199], D_l

Epoch [1/20], Step [136/1199], D_loss: 2.2146, G_loss: 4.0309
Epoch [1/20], Step [137/1199], D_loss: 0.7117, G_loss: 3.6278
Epoch [1/20], Step [138/1199], D_loss: 1.7887, G_loss: 2.8514
Epoch [1/20], Step [139/1199], D_loss: 0.4609, G_loss: 1.9610
Epoch [1/20], Step [140/1199], D_loss: 1.5474, G_loss: 4.5175
Epoch [1/20], Step [141/1199], D_loss: 0.0992, G_loss: 5.7475
Epoch [1/20], Step [142/1199], D_loss: 1.9227, G_loss: 4.6920
Epoch [1/20], Step [143/1199], D_loss: 1.3526, G_loss: 1.2442
Epoch [1/20], Step [144/1199], D_loss: 0.6224, G_loss: 1.8536
Epoch [1/20], Step [145/1199], D_loss: 1.1871, G_loss: 1.6999
Epoch [1/20], Step [146/1199], D_loss: 0.5346, G_loss: 2.7578
Epoch [1/20], Step [147/1199], D_loss: 1.7159, G_loss: 3.3555
Epoch [1/20], Step [148/1199], D_loss: 0.5910, G_loss: 2.2566
Epoch [1/20], Step [149/1199], D_loss: 0.2018, G_loss: 2.5982
Epoch [1/20], Step [150/1199], D_loss: 0.1759, G_loss: 2.0095
Epoch [1/20], Step [151/1199], D_loss: 0.5839, G_loss: 1.9942
Epoch [1

Epoch [1/20], Step [269/1199], D_loss: 0.5342, G_loss: 2.8987
Epoch [1/20], Step [270/1199], D_loss: 1.1228, G_loss: 0.3682
Epoch [1/20], Step [271/1199], D_loss: 0.6316, G_loss: 2.1123
Epoch [1/20], Step [272/1199], D_loss: 1.6698, G_loss: 1.9967
Epoch [1/20], Step [273/1199], D_loss: 0.7906, G_loss: 3.1426
Epoch [1/20], Step [274/1199], D_loss: 2.6105, G_loss: 1.7407
Epoch [1/20], Step [275/1199], D_loss: 3.4270, G_loss: 1.2429
Epoch [1/20], Step [276/1199], D_loss: 4.1024, G_loss: 2.4105
Epoch [1/20], Step [277/1199], D_loss: 1.2633, G_loss: 1.2778
Epoch [1/20], Step [278/1199], D_loss: 1.8010, G_loss: 1.4463
Epoch [1/20], Step [279/1199], D_loss: 0.8344, G_loss: 2.0387
Epoch [1/20], Step [280/1199], D_loss: 0.9174, G_loss: 2.5973
Epoch [1/20], Step [281/1199], D_loss: 1.3489, G_loss: 2.1199
Epoch [1/20], Step [282/1199], D_loss: 1.7132, G_loss: 3.5274
Epoch [1/20], Step [283/1199], D_loss: 1.6098, G_loss: 4.5581
Epoch [1/20], Step [284/1199], D_loss: 0.6706, G_loss: 5.1408
Epoch [1

Epoch [1/20], Step [403/1199], D_loss: 1.2216, G_loss: 1.4697
Epoch [1/20], Step [404/1199], D_loss: 1.5556, G_loss: 1.6190
Epoch [1/20], Step [405/1199], D_loss: 0.3672, G_loss: 1.8556
Epoch [1/20], Step [406/1199], D_loss: 1.4648, G_loss: 3.3628
Epoch [1/20], Step [407/1199], D_loss: 1.0455, G_loss: 4.0759
Epoch [1/20], Step [408/1199], D_loss: 0.2878, G_loss: 2.3147
Epoch [1/20], Step [409/1199], D_loss: 0.3303, G_loss: 3.0509
Epoch [1/20], Step [410/1199], D_loss: 0.7553, G_loss: 1.6096
Epoch [1/20], Step [411/1199], D_loss: 0.6740, G_loss: 0.4969
Epoch [1/20], Step [412/1199], D_loss: 0.7064, G_loss: 2.0962
Epoch [1/20], Step [413/1199], D_loss: 0.9050, G_loss: 1.9255
Epoch [1/20], Step [414/1199], D_loss: 1.7549, G_loss: 1.2144
Epoch [1/20], Step [415/1199], D_loss: 1.0477, G_loss: 3.0815
Epoch [1/20], Step [416/1199], D_loss: 0.3470, G_loss: 2.9607
Epoch [1/20], Step [417/1199], D_loss: 3.8093, G_loss: 0.8042
Epoch [1/20], Step [418/1199], D_loss: 0.5702, G_loss: 1.9973
Epoch [1

Epoch [1/20], Step [536/1199], D_loss: 2.8486, G_loss: 4.1682
Epoch [1/20], Step [537/1199], D_loss: 0.6078, G_loss: 4.3104
Epoch [1/20], Step [538/1199], D_loss: 1.6688, G_loss: 1.5270
Epoch [1/20], Step [539/1199], D_loss: 0.9776, G_loss: 5.0434
Epoch [1/20], Step [540/1199], D_loss: 3.6674, G_loss: 2.3987
Epoch [1/20], Step [541/1199], D_loss: 0.9715, G_loss: 2.7429
Epoch [1/20], Step [542/1199], D_loss: 0.4717, G_loss: 1.3416
Epoch [1/20], Step [543/1199], D_loss: 2.5131, G_loss: 1.6740
Epoch [1/20], Step [544/1199], D_loss: 1.0754, G_loss: 2.6051
Epoch [1/20], Step [545/1199], D_loss: 0.3972, G_loss: 0.2700
Epoch [1/20], Step [546/1199], D_loss: 1.1662, G_loss: 1.3231
Epoch [1/20], Step [547/1199], D_loss: 1.6021, G_loss: 3.8390
Epoch [1/20], Step [548/1199], D_loss: 0.4820, G_loss: 2.7195
Epoch [1/20], Step [549/1199], D_loss: 0.5573, G_loss: 3.5200
Epoch [1/20], Step [550/1199], D_loss: 1.7782, G_loss: 3.2111
Epoch [1/20], Step [551/1199], D_loss: 0.7439, G_loss: 3.1797
Epoch [1

...

Epoch [20/20], Step [617/1199], D_loss: 0.4106, G_loss: 0.6881
Epoch [20/20], Step [618/1199], D_loss: 1.6714, G_loss: 0.6034
Epoch [20/20], Step [619/1199], D_loss: 0.2411, G_loss: 0.7224
Epoch [20/20], Step [620/1199], D_loss: 1.7715, G_loss: 2.0074
Epoch [20/20], Step [621/1199], D_loss: 0.3120, G_loss: 0.3517
Epoch [20/20], Step [622/1199], D_loss: 2.1252, G_loss: 1.1589
Epoch [20/20], Step [623/1199], D_loss: 1.3665, G_loss: 1.0685
Epoch [20/20], Step [624/1199], D_loss: 1.0402, G_loss: 1.6728
Epoch [20/20], Step [625/1199], D_loss: 0.5877, G_loss: 0.8526
Epoch [20/20], Step [626/1199], D_loss: 1.1880, G_loss: 0.4865
Epoch [20/20], Step [627/1199], D_loss: 2.7277, G_loss: 1.6293
Epoch [20/20], Step [628/1199], D_loss: 2.0743, G_loss: 0.6773
Epoch [20/20], Step [629/1199], D_loss: 0.5610, G_loss: 1.1002
Epoch [20/20], Step [630/1199], D_loss: 0.8213, G_loss: 0.3027
Epoch [20/20], Step [631/1199], D_loss: 0.8251, G_loss: 1.9500
Epoch [20/20], Step [632/1199], D_loss: 0.9746, G_loss:

Epoch [20/20], Step [748/1199], D_loss: 1.2539, G_loss: 1.4019
Epoch [20/20], Step [749/1199], D_loss: 1.8726, G_loss: 0.7416
Epoch [20/20], Step [750/1199], D_loss: 0.8566, G_loss: 0.4672
Epoch [20/20], Step [751/1199], D_loss: 1.5432, G_loss: 0.6009
Epoch [20/20], Step [752/1199], D_loss: 0.7073, G_loss: 0.8156
Epoch [20/20], Step [753/1199], D_loss: 0.6053, G_loss: 1.9556
Epoch [20/20], Step [754/1199], D_loss: 1.0199, G_loss: 0.9402
Epoch [20/20], Step [755/1199], D_loss: 1.8073, G_loss: 0.6241
Epoch [20/20], Step [756/1199], D_loss: 1.1748, G_loss: 0.3691
Epoch [20/20], Step [757/1199], D_loss: 1.4804, G_loss: 0.5948
Epoch [20/20], Step [758/1199], D_loss: 0.5179, G_loss: 0.6962
Epoch [20/20], Step [759/1199], D_loss: 1.6445, G_loss: 1.2211
Epoch [20/20], Step [760/1199], D_loss: 2.5502, G_loss: 0.7906
Epoch [20/20], Step [761/1199], D_loss: 0.3469, G_loss: 0.6587
Epoch [20/20], Step [762/1199], D_loss: 1.8298, G_loss: 0.4615
Epoch [20/20], Step [763/1199], D_loss: 0.3479, G_loss:

Epoch [20/20], Step [880/1199], D_loss: 2.4522, G_loss: 1.5433
Epoch [20/20], Step [881/1199], D_loss: 1.5737, G_loss: 0.3946
Epoch [20/20], Step [882/1199], D_loss: 1.1972, G_loss: 0.3936
Epoch [20/20], Step [883/1199], D_loss: 0.6507, G_loss: 1.9984
Epoch [20/20], Step [884/1199], D_loss: 0.5797, G_loss: 0.6416
Epoch [20/20], Step [885/1199], D_loss: 0.4596, G_loss: 1.1781
Epoch [20/20], Step [886/1199], D_loss: 1.6914, G_loss: 1.0826
Epoch [20/20], Step [887/1199], D_loss: 0.2860, G_loss: 0.8444
Epoch [20/20], Step [888/1199], D_loss: 1.2589, G_loss: 1.0886
Epoch [20/20], Step [889/1199], D_loss: 0.9890, G_loss: 0.6897
Epoch [20/20], Step [890/1199], D_loss: 0.8174, G_loss: 1.5929
Epoch [20/20], Step [891/1199], D_loss: 1.2545, G_loss: 0.2347
Epoch [20/20], Step [892/1199], D_loss: 1.1690, G_loss: 0.2335
Epoch [20/20], Step [893/1199], D_loss: 0.7657, G_loss: 1.7149
Epoch [20/20], Step [894/1199], D_loss: 0.4588, G_loss: 0.4016
Epoch [20/20], Step [895/1199], D_loss: 2.1560, G_loss:

Epoch [20/20], Step [1011/1199], D_loss: 1.2118, G_loss: 1.3035
Epoch [20/20], Step [1012/1199], D_loss: 0.9018, G_loss: 1.1887
Epoch [20/20], Step [1013/1199], D_loss: 0.7854, G_loss: 1.4202
Epoch [20/20], Step [1014/1199], D_loss: 1.8034, G_loss: 0.5907
Epoch [20/20], Step [1015/1199], D_loss: 0.3294, G_loss: 0.6502
Epoch [20/20], Step [1016/1199], D_loss: 1.2602, G_loss: 1.6925
Epoch [20/20], Step [1017/1199], D_loss: 2.4952, G_loss: 0.7992
Epoch [20/20], Step [1018/1199], D_loss: 1.3028, G_loss: 0.3365
Epoch [20/20], Step [1019/1199], D_loss: 1.3369, G_loss: 0.4400
Epoch [20/20], Step [1020/1199], D_loss: 3.0512, G_loss: 0.8804
Epoch [20/20], Step [1021/1199], D_loss: 2.7800, G_loss: 1.1319
Epoch [20/20], Step [1022/1199], D_loss: 0.4908, G_loss: 0.6316
Epoch [20/20], Step [1023/1199], D_loss: 0.8465, G_loss: 1.0832
Epoch [20/20], Step [1024/1199], D_loss: 1.0169, G_loss: 1.4764
Epoch [20/20], Step [1025/1199], D_loss: 0.5350, G_loss: 0.8076
Epoch [20/20], Step [1026/1199], D_loss:

Epoch [20/20], Step [1141/1199], D_loss: 0.5536, G_loss: 1.0833
Epoch [20/20], Step [1142/1199], D_loss: 0.3229, G_loss: 0.7264
Epoch [20/20], Step [1143/1199], D_loss: 0.5066, G_loss: 0.8824
Epoch [20/20], Step [1144/1199], D_loss: 1.1128, G_loss: 0.6257
Epoch [20/20], Step [1145/1199], D_loss: 0.8720, G_loss: 0.6015
Epoch [20/20], Step [1146/1199], D_loss: 2.2632, G_loss: 0.9835
Epoch [20/20], Step [1147/1199], D_loss: 1.7264, G_loss: 0.1006
Epoch [20/20], Step [1148/1199], D_loss: 0.3629, G_loss: 1.2097
Epoch [20/20], Step [1149/1199], D_loss: 1.8474, G_loss: 0.8252
Epoch [20/20], Step [1150/1199], D_loss: 0.9887, G_loss: 1.1075
Epoch [20/20], Step [1151/1199], D_loss: 0.3134, G_loss: 0.2262
Epoch [20/20], Step [1152/1199], D_loss: 0.3671, G_loss: 0.4482
Epoch [20/20], Step [1153/1199], D_loss: 0.5265, G_loss: 0.8345
Epoch [20/20], Step [1154/1199], D_loss: 0.7209, G_loss: 0.7654
Epoch [20/20], Step [1155/1199], D_loss: 0.5442, G_loss: 1.3970
Epoch [20/20], Step [1156/1199], D_loss:

## Resources

- Much of the code in this notebook is modified from togheppi's implementation of [cDCGAN](https://github.com/togheppi/cDCGAN).